<a href="https://colab.research.google.com/github/sarthak395/Dass_Project/blob/main/Memory/Memorymodelextract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import numpy as np


In [ ]:
# LOADING DATA
def load_data(input_file):
    data = pd.read_csv(input_file)
    return data


In [ ]:
def inputcols(input_dataset):
    names = input_dataset['Name']
    details = input_dataset['Details']
    return names, details


In [ ]:
data = load_data('memory.csv')
names, details = inputcols(data)


In [ ]:
def extract_company_name(names,expression):
    companylist_name = []
    for name in names:
        re_search = re.search(expression, name)
        if re_search:
            companylist_name.append(name)
    return companylist_name

In [ ]:
def extract_company_details(details,expression):
    companylist_det = []
    for det in details:
        re_search = re.search(expression, det)
        if re_search:
            companylist_det.append(det)
    return companylist_det;

In [ ]:
def filtering_company_products(names, details, expression):
    companylist_name = extract_company_name(names, expression);
    companylist_det = extract_company_details(details, expression);
    companylist = pd.concat([pd.Series(companylist_name), pd.Series(companylist_det)], axis=1)
    return companylist;

In [ ]:
def extract_company_model_data(companylist,model_expression,company_name):
    companymodelnumbers = []

    for name, det in companylist.itertuples(index=False):
        re_search = re.search(model_expression, name)
        if re_search:
            companymodelnumbers.append({"company": company_name, "name": name, "model": re_search.group(0)})
        else:
            re_search2 = re.search(model_expression, det)
            if re_search2:
                companymodelnumbers.append({"company": company_name, "name": name, "model": re_search2.group(0)})
            else:
                companymodelnumbers.append({"company": company_name, "name": name, "model": np.nan})

    # NOW CHECK

    companymodelnumbers = pd.DataFrame(companymodelnumbers)
    return companymodelnumbers;

In [80]:
def enhance_model_with_size (size_expression , company_model_data):
    model_size = []
    for company ,name, model in company_model_data.itertuples(index=False):
        if pd.isnull(name):
            model_size.append({"name": name,"company":company,"model":model, "size": np.nan})
            continue
        re_search = re.search(size_expression, name)
        if re_search:
            model_size.append({"name": name,"company":company,"model":model, "size": re_search.group(0)})
        else:
            model_size.append({"name": name,"company":company,"model":model, "size": np.nan})

    model_size = pd.DataFrame(model_size)
    company_model_with_size_data = pd.merge(company_model_data, model_size, on=['name','company','model'], how='inner')
    return company_model_with_size_data;

In [81]:
def enhance_model_with_freq (freq_expression , company_model_data):
    model_freq = []
    for company,name, model in company_model_data.itertuples(index=False):
        if pd.isnull(name) :
            model_freq.append({"name": name,"company":company,"model":model, "freq": np.nan})
            continue
        re_search = re.search(freq_expression, name)
        if re_search:
            model_freq.append({"name": name,"company":company,"model":model, "freq": re_search.group(0)})
        else:
            model_freq.append({"name": name,"company":company,"model":model, "freq": np.nan})

    model_freq = pd.DataFrame(model_freq)
    company_model_with_freq_data = pd.merge(company_model_data, model_freq, on=['name','company','model'], how='left')
    return company_model_with_freq_data;

In [71]:
def complete_enhanced_data(company_model_with_size_data,company_model_with_freq_data):
    enhanced_data = pd.merge(company_model_with_size_data, company_model_with_freq_data, on=['name','company','model'], how='left')
    return enhanced_data;

In [ ]:
class Company:
    def __init__(self,expression,model_expression,names,details,company_name):
        self.company_name = company_name
        self.names = names;
        self.details = details;
        self.expression = expression;
        self.model_expression = model_expression;
        self.productlist = pd.DataFrame()
        self.model_data = pd.DataFrame()
        self.model_with_size = pd.DataFrame()
        self.model_with_freq = pd.DataFrame()
        self.enhanced_data = pd.DataFrame()
     
    def setcompanylist(self):
        self.productlist = filtering_company_products(self.names, self.details, self.expression)
    
    def setmodel_data(self):
        self.model_data = extract_company_model_data(self.productlist,self.model_expression,self.company_name)

    def setmodel_with_size(self,size_expression):
        self.model_with_size = enhance_model_with_size(size_expression,self.model_data)

    def setmodel_with_freq(self,freq_expression):
        self.model_with_freq = enhance_model_with_freq(freq_expression,self.model_data)
        
    def setenhanced_data(self):
        self.enhanced_data = complete_enhanced_data(self.model_with_size,self.model_with_freq)
    
    def callall(self,size_expression,freq_expression):
        self.setcompanylist()
        self.setmodel_data()
        self.setmodel_with_size(size_expression)
        self.setmodel_with_freq(freq_expression)
        self.setenhanced_data()
        return self.enhanced_data

In [82]:
adata = Company('(?i)adata ' , '(AX4U|AD4U|AX4S)[A-Za-z0-9-]*( )?' , names , details , "Adata")
crucial = Company('(?i)crucial ' , '(BL|CB|CT)[A-Za-z0-9-]*( )?' , names , details , "Crucial")
corcair = Company('(?i)corsair ' , '(CMD|CMK|CMZ|CMW|CMT|CMV)[A-Za-z0-9-]*( )?' , names , details , "Corcair")
gskill = Company('(?i)(g.skill) ' , '(F4-)[A-Za-z0-9-]*( )?' , names , details , "Gskill")
antec = Company('(?i)(antec) ' , '(AM)[A-Za-z0-9-]*( )?' , names , details , "Antec")
gigabyte =  Company('(?i)(gigabyte) ' , '(GP-)[A-Za-z0-9-]*( )?' , names , details , "Gigabyte")

size_expression = '[0-9]{1,2}( )?GB( )?'
freq_expression = '[0-9]{4}( )?(?i)MHz( )?'

adata.callall(size_expression,freq_expression)
crucial.callall(size_expression,freq_expression)
corcair.callall(size_expression,freq_expression)
gskill.callall(size_expression,freq_expression)
antec.callall(size_expression,freq_expression)
gigabyte.callall(size_expression,freq_expression)

result = pd.concat([adata.enhanced_data,crucial.enhanced_data,corcair.enhanced_data,gskill.enhanced_data,antec.enhanced_data,gigabyte.enhanced_data] ,axis=0, ignore_index=True)

result.to_csv('result.csv' , index=False)